# By-Band _g_-Point Reduction

# Dependencies

`numpy` is installed in the Python environment at NERSC (`module load python`), but `xarray` is not, so the user must install the package on their own. `PIPPATH` is the assumed location. This notebook depends heavily on `xarray`.

In [1]:
import os, sys, shutil, glob

# "standard" install
import numpy as np

from multiprocessing import Pool

# directory in which libraries installed with conda are saved
PIPPATH = '{}/.local/'.format(os.path.expanduser('~')) + \
    'cori/3.7-anaconda-2019.10/lib/python3.7/site-packages'
PATHS = ['common', PIPPATH]
for path in PATHS: sys.path.append(path)

# needed at AER unless i update `pandas`
import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# user must do `pip install xarray` on cori (or other NERSC machines)
import xarray as xa

# local module
import by_band_lib as BYBAND

# Static Inputs

In [2]:
# only do one domain or the other
DOLW = True
DOSW = not DOLW
DOMAIN = 'LW' if DOLW else 'SW'
NBANDS = 16 if DOLW else 14

# forcing scenario (0 is no forcing...need a more comprehensive list)
IFORCING = 0

# does band-splitting need to be done, or are there existing files 
# that have divided up the full k-distribution?
BANDSPLIT = False

# remove the netCDFs that are generated for all of the combinations 
# and iterations of combinations in bandOptimize()
CLEANUP = True

# cost function variables
CFCOMPS = ['flux_net', 'heating_rate', 'band_flux_net']
CFLEVS = [0, 10000, 102000] # pressure levels of interest in Pa
CFWGT = [1., 1., 1., .5]

# Paths

In [3]:
PROJECT = '/project/projectdirs/e3sm/pernak18/reference_netCDF/g-point-reduce'
KFULLNC = '{}/rrtmgp-data-lw-g256-2018-12-04.nc'.format(PROJECT)
EXE = 'garand_atmos/rrtmgp_garand_atmos'
GARAND = '{}/multi_garand_template_single_band.nc'.format(PROJECT)

# test (RRTMGP) and reference (LBL) flux netCDF files
TESTNC = '{}/rrtmgp-lw-flux-inputs-outputs-garandANDpreind.nc'.format(PROJECT)
REFNC = '{}/lblrtm-lw-flux-inputs-outputs-garandANDpreind.nc'.format(PROJECT)
PATHS = [KFULLNC, EXE, TESTNC, REFNC, GARAND]

BANDSPLITDIR = 'band_k_dist'
FULLBANDFLUXDIR = 'full_band_flux'

for PATH in PATHS: BYBAND.pathCheck(PATH)

CWD = os.getcwd()

# Band Splitting

Break up full _k_-distribution file into separate distributions for each band, then calculate the corresponding fluxes. This should only need to be run once.

After some clarifications from Robert (30-Nov-2020), I believe the plan of action is:

1. Perform _g_-point combination for 1 band at a time
2. Combine condensed _k_-distribution for a given band with the full distributions from the rest of the bands
3. Run RRTMGP on output from item 2
4. Compute cost function for broadband fluxes, not by-band

Original Plan of Action:

- [] We would create Nbands k-distribution files
- [] We’d use Python to drive the Fortran executable Nbands times to produce Nbands flux results
- [] The trial g-point combinations then loop over bands and the possible g-point combinations within each band, creating k-distribution and band-wise flux files for each possible combination
- [] The Python code assembles broadband fluxes from the band-wise flux files in order to compute the cost functions

In [4]:
if BANDSPLIT:
    print('Band splitting commenced')
    BYBAND.pathCheck(BANDSPLITDIR, mkdir=True)
    BYBAND.pathCheck(FULLBANDFLUXDIR, mkdir=True)
    kFiles, fullBandFluxes = [], []
    for iBand in range(NBANDS):
        # divide full k-distribution into subsets for each band
        kObj = BYBAND.gCombine_kDist(KFULLNC, iBand, DOLW, 
            fullBandKDir=BANDSPLITDIR, fullBandFluxDir=FULLBANDFLUXDIR, 
            cleanup=CLEANUP)
        kFiles.append(kObj.kBandNC)
        kObj.kDistBand()

        # quick, non-parallelized flux calculations (because the 
        # executable is run in one directory)
        kObj.fluxCompute(kObj.kBandNC)
        fullBandFluxes.append(kObj.fluxBandNC)
    # end band loop
    print('Band splitting completed')
else:
    kFiles = sorted(glob.glob('{}/coefficients_{}_band??.nc'.format(
        BANDSPLITDIR, DOMAIN)))
    fullBandFluxes = sorted(glob.glob('{}/flux_{}_band??.nc'.format(
        FULLBANDFLUXDIR, DOMAIN)))

    if len(kFiles) == 0 or len(fullBandFluxes) == 0:
        print('WARNING: set `BANDSPLIT` to `True` and run this cell again')
# endif BANDSPLIT


# _g_-Point Combining

Combine _g_-point reduced for bands with full-band fluxes from other bands, find optimal _g_-point combination for given iteration, proceed to next iteration.

In [5]:
kBandDict = {}
for iBand, kFile in enumerate(kFiles):
    #if iBand != 3: continue
    band = iBand + 1
    iterBand = 1
    kObj = BYBAND.gCombine_kDist(kFile, iBand, DOLW, 
        exeRRTMGP=EXE, fullBandKDir=BANDSPLITDIR, 
        fullBandFluxDir=FULLBANDFLUXDIR, cleanup=CLEANUP)
    kObj.gPointCombine()
    kBandDict['band{:02d}'.format(band)] = kObj

    print('Band {} complete'.format(band))
# end kFile loop

"""
BYBAND.gCombine_Cost(kBandDict, REFNC, TESTNC, 
            iForce, iterBand, profilesNC=GARAND, exeRRTMGP=EXE, 
            fullBandKDir=BANDSPLITDIR, fullBandFluxDir=FULLBANDFLUXDIR, 
            costFuncComp=['flux_net', 'band_flux_net'], 
            cleanup=CLEANUP)
"""

def optCostFunc(kBandFile, iBand, doLW, iForce, fluxFiles, cleanup=False):
    """
    needs a lot of work -- just spitballin

    single-band optimization, to be combined with broadband fluxes 
    from other bands before cost function optimization 
    """

    band = iBand + 1
    iterBand = 1
    while True:
        print('Starting Band {}, iteration {}'.format(band, iterBand))
        # start with `kFile` with no g-point combinations for a given band
        kObj = BYBAND.kDistOptBand(kBandFile, REFNC, TESTNC, 
            iBand, doLW, iForce, iterBand, profilesNC=GARAND, exeRRTMGP=EXE, 
            fullBandKDir=BANDSPLITDIR, fullBandFluxDir=FULLBANDFLUXDIR, 
            costFuncComp=['flux_net', 'band_flux_net'], 
            cleanup=CLEANUP)

        # combine g-points in band and generate corresponding netCDF
        print('\tCombining g-points and writing associated netCDF files')
        kObj.gPointCombine()
        break
# end band

Band 1 complete
Band 2 complete
Band 3 complete
Band 4 complete
Band 5 complete
Band 6 complete
Band 7 complete
Band 8 complete
Band 9 complete
Band 10 complete
Band 11 complete
Band 12 complete
Band 13 complete
Band 14 complete
Band 15 complete
Band 16 complete
